In [26]:
# Install required packages.
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [40]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import networkx as nx

import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv, SAGEConv
from torch_geometric.transforms import RandomLinkSplit
from sklearn.metrics.pairwise import cosine_similarity
from torch_geometric import seed_everything
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder


In [41]:
#ESTO ES PARA EJECUTAR EN COLAB
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
# Ruta donde guardaste el archivo gpickle
ruta_guardado = '/content/drive/My Drive/Colab Notebooks/GRAFO RETO/grafo.pkl'

# Carga el grafo desde el archivo gpickle
with open(ruta_guardado, 'rb') as file:
    G7 = pickle.load(file)

In [66]:
#Numero de features
features=list(G7.nodes(data=True))[0][1]
print(features)
num_features = len(features['feature'])
print('Numero de atributos:',num_features)

{'feature': [7, 'B11730', 'warm', 1.1, 'Formal', 1, 'smooth', 'street', 'tight', 'freetime'], 'label': 0}
Numero de atributos: 10


In [67]:
# Obtener nodos y características de nodos del grafo
nodos = list(G7.nodes)
conexion = list(G7.edges)
caracteristicas_nodos = [G7.nodes[nodo] for nodo in nodos]

In [68]:
df_caracteristicas_nodos = pd.DataFrame(caracteristicas_nodos, index=nodos)
df_caracteristicas_nodos.head()

,feature,label
0,"[7, B11730, warm, 1.1, Formal, 1, smooth, stre...",0
1,"[7, ECE9D6, warm, 3.0, Informal, 2, horizontal...",1
2,"[7, 5F5E5E, warm, 2.0, Informal, 1, smooth, cl...",2
3,"[7, 00008b, warm, 2.0, Informal, 1, horizontal...",3
4,"[7, 000000, cold, 3.1, Informal, 3, sheets, ca...",4


In [69]:
caracteristicas_df = df_caracteristicas_nodos['feature'].apply(pd.Series)
caracteristicas_df.columns = ['season','color','weather','subnivel','formalidad','adventurous','estampado','estilo','fit','application']
caracteristicas_df.head()

,season,color,weather,subnivel,formalidad,adventurous,estampado,estilo,fit,application
0,7,B11730,warm,1.1,Formal,1,smooth,street,tight,freetime
1,7,ECE9D6,warm,3.0,Informal,2,horizontal_stripes,classic,straight,work
2,7,5F5E5E,warm,2.0,Informal,1,smooth,classic,straight,freetime
3,7,00008b,warm,2.0,Informal,1,horizontal_stripes,casual,straight,freetime
4,7,000000,cold,3.1,Informal,3,sheets,casual,loose,freetime


In [70]:
df_caracteristicas_nodos = pd.concat([df_caracteristicas_nodos.drop(columns=['feature']), caracteristicas_df], axis=1)
df_caracteristicas_nodos = df_caracteristicas_nodos.drop(columns=['season'])
df_caracteristicas_nodos.head()

,label,color,weather,subnivel,formalidad,adventurous,estampado,estilo,fit,application
0,0,B11730,warm,1.1,Formal,1,smooth,street,tight,freetime
1,1,ECE9D6,warm,3.0,Informal,2,horizontal_stripes,classic,straight,work
2,2,5F5E5E,warm,2.0,Informal,1,smooth,classic,straight,freetime
3,3,00008b,warm,2.0,Informal,1,horizontal_stripes,casual,straight,freetime
4,4,000000,cold,3.1,Informal,3,sheets,casual,loose,freetime


In [72]:
df_caracteristicas_nodos.dtypes

,0
label,int64
color,object
weather,object
subnivel,float64
formalidad,object
adventurous,int64
estampado,object
estilo,object
fit,object
application,object


In [89]:
for col in df_caracteristicas_nodos.columns:
    print(f'{col}:{len(df_caracteristicas_nodos[col].unique())}')

label:1923
color:38
weather:2
subnivel:6
formalidad:2
adventurous:5
estampado:25
estilo:6
fit:5
application:5


In [73]:
categoricas = df_caracteristicas_nodos.select_dtypes(exclude=['int']).columns
categoricas

Index(['color', 'weather', 'subnivel', 'formalidad', 'estampado', 'estilo',
       'fit', 'application'],
      dtype='object')

In [80]:
# Crear un LabelEncoder para cada columna categórica
df_codificado = pd.get_dummies(df_caracteristicas_nodos, columns=categoricas,drop_first=True)
df_codificado.head()
columnas = df_codificado.columns
df_codificado[columnas] = df_codificado[columnas].astype(int)
df_codificado.head()

,label,adventurous,color_000081,color_00008b,color_0000FF,color_008000,color_153668,color_164A0A,color_40E0D0,color_462C0E,...,estilo_minimal,estilo_night,estilo_street,fit_oversize,fit_straight,fit_tight,application_night,application_special_occasion,application_work,application_working_girl
0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,3,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
df_codificado.values

array([[   0,    1,    0, ...,    0,    0,    0],
       [   1,    2,    0, ...,    0,    1,    0],
       [   2,    1,    0, ...,    0,    0,    0],
       ...,
       [1920,    3,    0, ...,    0,    0,    0],
       [1921,    1,    0, ...,    0,    0,    0],
       [1922,    1,    0, ...,    0,    0,    0]])

In [82]:
x_tensor = torch.tensor(df_codificado.drop(columns=['label']).values, dtype=torch.float32)
x_tensor.shape

torch.Size([1923, 80])

In [83]:
x_tensor

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [2., 0., 0.,  ..., 0., 1., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [3., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.]])

In [85]:
edges=list(G7.edges)
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
print(edge_index,edge_index.shape)

tensor([[   0,    0,    0,  ..., 1908, 1908, 1914],
        [  53,   68,   91,  ..., 1910, 1912, 1917]]) torch.Size([2, 150028])


In [87]:
G7_data=Data(x=x_tensor, edge_index=edge_index)
G7_data

Data(x=[1923, 80], edge_index=[2, 150028])